# TrackChain - Track Parameters

In [1]:
### IMPORTS ###

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import datetime
import os
from tqdm.notebook import tqdm
import math

In [2]:
### ------------------------------ ###
### -------- DIRECTORIES --------- ###
### ------------------------------ ###

dir_src = os.path.abspath('')
dir_root = os.path.dirname(dir_src)
dir_data = os.path.join(dir_root, 'data')
dir_plots = os.path.join(dir_root, 'plots')

In [3]:
### ------------------------------ ###
### ----------- VALUES ----------- ###
### ------------------------------ ###

rho_gon = np.pi / 200 # Conversion factor from gon to radians
rho_deg = np.pi / 180 # Conversion factor from degree to radians

sigma_dist_default = 50 # [mm]
sigma_tilt_default = 0.072 # [deg]
sigma_tilt_default_rad = sigma_tilt_default * rho_deg # [rad]

num_reps = int(1e6) # Number of repetitions for Monte Carlo simulation

h0 = 0 # Height level of first point

nominal_dist_lateral = 4.8

sbb_profile_dist = 4.8
width_track = 1.435 # [m] width of a SBB track

In [27]:
def monte_carlo_track_params(
    sigma_tilt:float,
    sigma_dist:float,
    dist_tot:float=9.6,
    nominal_dist:float=1.2,
    nominal_dist_lat:float=4.8,
    single_chain:bool=True) -> tuple[np.ndarray, np.ndarray]:
    '''
    Analysis of precision using a Monte Carlo simulation

    Parameters
    ----------
    num_sensors : Integer
        Number of sensors used
    sigma_tilt : Float
        Standard deviation for the tilt angles
    sigma_dist : Float
        Standard deviation for the distances
    nominal_dist : Float, optional
        Nominal distance between any two points

    Returns
    -------
    std_open : np.ndarray
        Array of standard deviations per point with one end fixed
    std_closed : np.ndarray
        Array of standard deviations per point with both ends fixed
    std_open_lat : np.ndarray
        Array of standard deviations per point on the second track with one end fixed
    std_closed_lat : np.ndarray
        Array of standard deviations per point on the second track with both ends fixed
    '''

    num_sensors = int(dist_tot // nominal_dist)
    num_sensors_lat = int(dist_tot // nominal_dist_lat) + 1
    num_points = num_sensors + 1

    # Create random observations for the distances and the angles
    r_dist = np.random.normal(nominal_dist * 1000, sigma_dist, (num_sensors, num_reps))
    r_ang = np.random.normal(0, sigma_tilt * rho_deg, (num_sensors, num_reps))

    # Height differences per point
    dh = r_dist * np.sin(r_ang)

    # Cumulative sum of height differences, last point unknown
    h = np.insert(np.cumsum(dh, axis=0), 0, np.zeros(num_reps), axis=0)
    h_sbb = h[::int(sbb_profile_dist//nominal_dist)]
    
    if single_chain:
        r_dist_sec = np.random.normal(width_track * 1000, sigma_dist, (num_sensors_lat, num_reps))
        r_ang_sec = np.random.normal(0, sigma_tilt * rho_deg, (num_sensors_lat, num_reps))
        
        dh_sec = r_dist_sec * np.sin(r_ang_sec)
        h_sec = h[::math.ceil(num_points / num_sensors_lat)] + dh_sec
        h_sbb_sec = h_sec[::int(sbb_profile_dist//nominal_dist_lat)]
    else:
        r_dist_sec = np.random.normal(nominal_dist * 1000, sigma_dist, (num_sensors, num_reps))
        r_ang_sec = np.random.normal(0, sigma_tilt * rho_deg, (num_sensors, num_reps))

        # Height differences per point
        dh_sec = r_dist_sec * np.sin(r_ang_sec)

        # Cumulative sum of height differences, last point unknown
        h_sec = np.insert(np.cumsum(dh_sec, axis=0), 0, np.zeros(num_reps), axis=0)
        h_sbb_sec = h_sec[::int(sbb_profile_dist//nominal_dist)]
    
    std_h_sbb = np.std(h_sbb, axis=1)
    
    h_sbb_avg = np.mean(np.array([h_sbb, h_sbb_sec]), axis=0)

    # Vertikale Pfeilhöhe
    pf_v = h_sbb[1:-1] - (h_sbb[:-2] + h_sbb[2:]) / 2

    # Überhöhung
    u = h_sbb - h_sbb_sec
    

    # Verwindung
    n = (u[1:] - u[:-1]) / sbb_profile_dist
    
    print(h_sbb[:, 0], pf_v[:, 0], u[:,0])
    
    std_pf_v = np.std(pf_v, axis=1)
    std_n = np.std(n, axis=1)

    return std_n, std_pf_v

monte_carlo_track_params(sigma_tilt_default, sigma_dist_default, nominal_dist=1.2)

[ 0.          8.94750517 11.32096291] [3.28702372] [-1.62055846  1.71275512  1.27206186]


(array([0.61748766, 0.76173777]), array([2.13350482]))